In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Dados Fictícios de Exemplo (Substitua pela sua base do M17/M20) ---
# Usarei uma base representativa para que o código seja executável.
data = {
    'Age': np.random.randint(20, 60, 100),
    'Gender': np.random.choice(['Male', 'Female'], 100),
    'Income': np.random.randint(30000, 150000, 100),
    'Education': np.random.choice(["Bachelor's Degree", "Master's Degree", "Doctorate"], 100),
    'Marital_Status': np.random.choice(['Single', 'Married'], 100),
    'Number_of_Children': np.random.randint(0, 4, 100),
    'Home_Ownership': np.random.choice(['Owned', 'Rented'], 100),
    'Credit_Score': np.random.choice(['High', 'Average', 'Low'], 100, p=[0.7, 0.2, 0.1]) # Base desbalanceada
}
df = pd.DataFrame(data)

# Pré-processamento e Encoding (Módulo 17/20)
df.columns = df.columns.str.lower().str.replace(' ', '_')
colunas_categoricas = ['gender', 'education', 'marital_status', 'home_ownership']
df_encoded = pd.get_dummies(df, columns=colunas_categoricas, drop_first=True)

# Definição de X e y
X = df_encoded.drop('credit_score', axis=1)
y = df_encoded['credit_score']

# Divisão do dataset (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalonamento dos dados (Regressão Logística é sensível à escala)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Balanceamento com SMOTE (Apenas na base de Treino)
smote = SMOTE(random_state=42)
X_train_balanceado, y_train_balanceado = smote.fit_resample(X_train_scaled, y_train)

print(f"Shape de Treino balanceado: {X_train_balanceado.shape}")
print("Contagem do Target Balanceado:")
print(y_train_balanceado.value_counts().to_markdown())

# Instanciar o modelo de Regressão Logística (multinomial para lidar com 3 classes)
# max_iter é aumentado para garantir convergência
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

# Treinar o modelo na base balanceada e escalonada
log_reg.fit(X_train_balanceado, y_train_balanceado)

# Fazer as previsões na base de TESTE (escalonada)
y_pred_test = log_reg.predict(X_test_scaled)

print("Modelo de Regressão Logística treinado e previsões realizadas com sucesso na base de Teste.")

accuracy = accuracy_score(y_test, y_pred_test)

print(f"Acurácia (Accuracy) na Base de Teste: {accuracy:.4f}")

cm = confusion_matrix(y_test, y_pred_test)
labels = sorted(y_test.unique())

print("Matriz de Confusão:")
cm_df = pd.DataFrame(cm, index=labels, columns=labels)
print(cm_df.to_markdown())

# Visualização da Matriz de Confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='PuBu',
            xticklabels=labels, yticklabels=labels)
plt.title('Matriz de Confusão (Regressão Logística)')
plt.ylabel('Valores Reais')
plt.xlabel('Previsões do Modelo')
plt.show()

report = classification_report(y_test, y_pred_test, output_dict=True, zero_division=0)
report_df = pd.DataFrame(report).transpose().round(4)

print("Relatório de Classificação (Precision, Recall, F1-Score, Support):")
print(report_df.to_markdown())

# Acurácia por Classe (Precision) e Recall Médio (Média Ponderada)
precision_macro = report_df.loc['macro avg', 'precision']
recall_macro = report_df.loc['macro avg', 'recall']

print(f"\nPrecision Médio (Macro Average): {precision_macro:.4f}")
print(f"Recall Médio (Macro Average): {recall_macro:.4f}")

# Obter nomes das features (após o One-Hot Encoding)
feature_names = X.columns.tolist()

# Obter classes (para Regressão Logística Multiclasse)
classes = log_reg.classes_

# Criar um DataFrame com os coeficientes
# log_reg.coef_ tem shape (n_classes, n_features)
coef_df = pd.DataFrame(log_reg.coef_, columns=feature_names, index=classes)

print("--- Coeficientes do Modelo (Impacto das Features) ---")
print("Quanto maior o valor (em módulo), maior o impacto da feature na previsão da classe.")
print(coef_df.T.to_markdown())

### Conclusão e Comparação de Desempenho

**Comparação com Naive Bayes (Módulo 20):**

A Regressão Logística geralmente oferece um desempenho mais equilibrado do que o Naive Bayes neste tipo de problema. Enquanto o Naive Bayes é mais rápido, ele sofre com a suposição de independência das *features*. A Regressão Logística, por ser mais robusta, tende a ter um **Recall e Precision mais altos** nas classes minoritárias ('Low' e 'Average'), que são cruciais para a gestão de risco.

**Justificativa de Escolha:**

A Regressão Logística é uma das melhores escolhas para o Credit Score porque:

1.  **Alta Interpretabilidade:** Os coeficientes nos permitem entender **quais fatores (Renda, Idade, Educação)** e com qual **peso** estão influenciando a decisão do Score, fornecendo *insights* valiosos para o negócio.
2.  **Saída Probabilística:** O modelo fornece a probabilidade de um cliente pertencer a cada classe (`High`, `Average`, `Low`), permitindo à instituição financeira definir um limite de risco mais flexível.
3.  **Desempenho Estável:** Após o escalonamento e balanceamento, a Regressão Logística oferece um bom equilíbrio entre precisão geral e a capacidade de identificar corretamente as classes de alto e baixo risco.